<a href="https://colab.research.google.com/github/Hana20-ai/deep_learning_with_Colab/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import tensorflow as tf
import os,datetime
import tensorflow_datasets as tfds

#import data from tensorflow_datasets 
data,info = tfds.load ( 'patch_camelyon', with_info = True, as_supervised = True)


In [ ]:
#Récuperer les données d'entrainement, de test et de validation dans le dataset 
train_data = data['train']
valid_data = data['validation']
test_data = data['test']

In [ ]:
#Normalisation des données (les images) : Prétraitement des images 
#fonction de prétraitement d'UNE image 
def pretrait(image, labels):
  image = tf.cast(image, tf.float32)
  #les pixels seront de format 0 et 1 en virgule flottante
  image /= 255.0  
  return image, labels

#Appliquer le prétraitement sur les ensembles de données en utilisant la fonction map()
train_data = train_data.map(pretrait)
valid_data = valid_data.map(pretrait)
test_data = test_data.map(pretrait)



#normalisation par lots en utilisant la fonction prefetch 
batch_size = 128 
train_data = train_data.batch(batch_size).prefetch(1)
valid_data = valid_data.batch(batch_size).prefetch(1)
test_data = test_data.batch(batch_size).prefetch(1)


#Séparer les images et leurs étiquettes lot par lot  pour les trois sous ensembles de données (appren supervisé) 
train_images, train_labels = next(iter(train_data))
valid_images, valid_labels = next(iter(valid_data))
test_images, test_labels  = next(iter(test_data))


In [ ]:
from tensorflow.keras import Sequential #le modele utilisé est séquentiel
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout #les couches CNN 
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers 

#création d'un réseau de neurones avec n couches 
model = Sequential([
                    Conv2D(256, 3,padding='same', kernel_initializer='he_uniform', activation='relu', input_shape = [96, 96, 3]),
                    MaxPooling2D(2),

                    Conv2D(256, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),

                    Conv2D(512, 3,padding='same',kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),

                    Conv2D(512, 3,padding='same',kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),

                    Conv2D(1024, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),

                    Conv2D(1024, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),

                    Flatten(),

                    #output layers: fully connected layers 
                    Dense(1024,kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(512,kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(256,kernel_initializer='he_uniform',activation = 'relu'),

                    Dense(1, activation = 'sigmoid'),
                    
                    ])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 96, 96, 256)       7168      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 48, 48, 256)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 48, 48, 256)       590080    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 24, 24, 256)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 24, 24, 512)       1180160   
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 12, 12, 512)     

In [ ]:
#Compiler le modèle:

model.compile (loss='binary_crossentropy',      #entropie binaire car les labels sont des (0,1)
               optimizer=optimizers.Adam(1e-4), #la descente de gradient
               metrics=['accuracy'])            #evaluer selon le taux de bonne prédiction 

#commencer l'appretissage
#epochs: nb passages sur l’ensemble des exemples de la base d’apprentissage lors de la descente de gradient
Appren = model.fit( train_images, train_labels, epochs = 100, validation_data = (valid_images, valid_labels), verbose=2)


Epoch 1/100
4/4 - 75s - loss: 3.2859 - accuracy: 0.7031 - val_loss: 3.4275 - val_accuracy: 0.4141 - 75s/epoch - 19s/step
Epoch 2/100
4/4 - 70s - loss: 1.3128 - accuracy: 0.6250 - val_loss: 0.8101 - val_accuracy: 0.5859 - 70s/epoch - 17s/step
Epoch 3/100
4/4 - 67s - loss: 0.6623 - accuracy: 0.5703 - val_loss: 0.6402 - val_accuracy: 0.5859 - 67s/epoch - 17s/step
Epoch 4/100
4/4 - 66s - loss: 0.5671 - accuracy: 0.7344 - val_loss: 0.6436 - val_accuracy: 0.7109 - 66s/epoch - 17s/step
Epoch 5/100
4/4 - 70s - loss: 0.5296 - accuracy: 0.7891 - val_loss: 0.6627 - val_accuracy: 0.6562 - 70s/epoch - 17s/step
Epoch 6/100
4/4 - 67s - loss: 0.4596 - accuracy: 0.8203 - val_loss: 0.6449 - val_accuracy: 0.6094 - 67s/epoch - 17s/step
Epoch 7/100
4/4 - 68s - loss: 0.4178 - accuracy: 0.8594 - val_loss: 0.6680 - val_accuracy: 0.7031 - 68s/epoch - 17s/step
Epoch 8/100
4/4 - 66s - loss: 0.3693 - accuracy: 0.8359 - val_loss: 0.6318 - val_accuracy: 0.6953 - 66s/epoch - 17s/step
Epoch 9/100
4/4 - 67s - loss: 0.

In [ ]:
#Tester le modèle 
eval= model.evaluate(test_images, test_labels)
eval

4/4 [==============================] - 12s 3s/step - loss: 1.4400 - accuracy: 0.7969


[1.440037488937378, 0.796875]